提取故事内容 + 初步得到采录者信息和地点

In [59]:
import os
import re
import csv
from tqdm import tqdm  # 导入 tqdm 库

# 定义文件路径
input_file_path = "/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/data/中国民间故事集成 山东_青海_内蒙古_黑龙江/中国民间故事集成青海卷.txt"
output_dir = "/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250120OCR/青海"
csv_file_path = os.path.join(output_dir, "stories_info.csv")

number = 667
# 创建输出目录
os.makedirs(output_dir, exist_ok=True)

# 读取文件内容
with open(input_file_path, "r", encoding="utf-8") as file:
    content = file.read()

# 正则表达式匹配标题行（编号 + 可能存在的空格 + 分隔符 + 标题）
title_pattern = re.compile(r"(\d{3})\s*[•.。](.*?)\n")
# 正则表达式匹配地点信息
location_pattern = re.compile(r"（(.*?)）")
# 正则表达式匹配讲述者信息
narrator_pattern = re.compile(r"讲\s*述\s*者[^\w\s]*(.*?)(?=\d{4}|$)", re.DOTALL)
# 正则表达式匹配采录者信息
recorder_pattern = re.compile(r"采\s*录\s*者[^\w\s]*(.*?)(?=\d{4}|$)", re.DOTALL)
# 正则表达式匹配讲述并记录信息
narrator_recorder_pattern = re.compile(r"讲\s*述\s*并\s*记\s*录[^\w\s]*(.*?)(?=\d{4}|$)", re.DOTALL)

# 找到所有标题行的位置
title_matches = list(title_pattern.finditer(content))

# 初始化存储结果的列表
stories_data = []

# 遍历 001 到 725，逐个定位标题行
for story_number in tqdm(range(1, number + 1), desc="处理故事", unit="个"):
    # 格式化编号为 3 位数（如 001, 002, ..., 725）
    story_number_str = f"{story_number:03d}"
    
    # 构造正则表达式，匹配当前编号的标题行
    current_title_pattern = re.compile(rf"{story_number_str}\s*[•.。](.*?)\n")
    title_match = current_title_pattern.search(content)
    
    if not title_match:
        # 如果没有找到当前编号的标题行，跳过
        continue

    # 提取标题
    story_title = title_match.group(1).strip()
    story_name = f"{story_number_str}{story_title}"

    # 提取地点信息
    location_match = location_pattern.search(content[title_match.end():])
    location = location_match.group(1).strip() if location_match else "NaN"

    # 提取故事内容
    if story_number < number:
        # 查找下一个标题行
        next_title_pattern = re.compile(rf"{story_number + 1:03d}\s*[•.。](.*?)\n")
        next_title_match = next_title_pattern.search(content[title_match.end():])
        if next_title_match:
            # 当前故事的内容是从当前标题开始到下一个标题之前
            story_content = content[title_match.start():title_match.end() + next_title_match.start()].strip()
        else:
            # 如果没有找到下一个标题行，则跳过
            continue
    else:
        # 最后一个故事的内容是从当前标题开始到文件末尾
        story_content = content[title_match.start():].strip()

    # 提取信息（优先级：讲述者 > 采录者 > 讲述并记录）
    narrator_match = narrator_pattern.search(story_content)
    recorder_match = recorder_pattern.search(story_content)
    narrator_recorder_match = narrator_recorder_pattern.search(story_content)

    if narrator_match:
        info_text = narrator_match.group(1).strip()  # 提取“讲述者”之后的内容
        info_type = "讲述者"
        # 去除多余的标点符号和特殊字符，只保留汉字和数字
        info_text = re.sub(r"[^\u4e00-\u9fa5\d]", "", info_text)
        # 剔除讲述者信息
        story_content = story_content[:narrator_match.start()].strip()
    elif recorder_match:
        info_text = recorder_match.group(1).strip()  # 提取“采录者”之后的内容
        info_type = "采录者"
        # 去除多余的标点符号和特殊字符，只保留汉字和数字
        info_text = re.sub(r"[^\u4e00-\u9fa5\d]", "", info_text)
        # 剔除采录者信息
        story_content = story_content[:recorder_match.start()].strip()
    elif narrator_recorder_match:
        info_text = narrator_recorder_match.group(1).strip()  # 提取“讲述并记录”之后的内容
        info_type = "讲述并记录"
        # 去除多余的标点符号和特殊字符，只保留汉字和数字
        info_text = re.sub(r"[^\u4e00-\u9fa5\d]", "", info_text)
        # 剔除讲述并记录信息
        story_content = story_content[:narrator_recorder_match.start()].strip()
    else:
        info_text = "NaN"
        info_type = "NaN"

    # 保存故事内容到单独的txt文件
    story_file_path = os.path.join(output_dir, f"{story_name}.txt")
    with open(story_file_path, "w", encoding="utf-8") as story_file:
        story_file.write(story_content)

    # 将故事信息添加到列表中
    stories_data.append({
        "故事编号": story_number_str,
        "故事标题": story_title,
        "地点信息": location,
        "采录者信息": info_text if info_type != "NaN" else "NaN"
    })

# 将故事信息保存到CSV文件
with open(csv_file_path, "w", encoding="utf-8", newline="") as csv_file:
    fieldnames = ["故事编号", "故事标题", "地点信息", "采录者信息"]
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(stories_data)

print(f"\n处理完成！结果已保存到 {output_dir} 目录中。")

处理故事: 100%|██████████| 667/667 [00:00<00:00, 2292.73个/s]


处理完成！结果已保存到 /Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250120OCR/青海 目录中。


内蒙古的有点特殊

In [69]:
import os
import re
import csv
from tqdm import tqdm  # 导入 tqdm 库

# 定义文件路径
input_file_path = "/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/data/中国民间故事集成 山东_青海_内蒙古_黑龙江/中国民间故事集成内蒙古卷.txt"
output_dir = "/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250120OCR/内蒙古"
csv_file_path = os.path.join(output_dir, "stories_info.csv")

number = 641
# 创建输出目录
os.makedirs(output_dir, exist_ok=True)

# 读取文件内容
with open(input_file_path, "r", encoding="utf-8") as file:
    content = file.read()

# 正则表达式匹配标题行（编号 + 可能存在的空格 + 分隔符 + 标题）
title_pattern = re.compile(r"(\d{3})\s*[•.。](.*?)\n")
# 正则表达式匹配地点信息
location_pattern = re.compile(r"（(.*?)）")
# 正则表达式匹配讲述者信息
narrator_pattern = re.compile(r"讲\s*述\s*者[^\w\s]*(.*?)(?=\d{4}|$)", re.DOTALL)
# 正则表达式匹配采录者信息
recorder_pattern = re.compile(r"采\s*录\s*者[^\w\s]*(.*?)(?=\d{4}|$)", re.DOTALL)
# 正则表达式匹配讲述并记录信息
narrator_recorder_pattern = re.compile(r"讲\s*述\s*并\s*记\s*录[^\w\s]*(.*?)(?=\d{4}|$)", re.DOTALL)
# 正则表达式匹配翻译者信息
translator_pattern = re.compile(r"翻\s*译\s*者[^\w\s]*(.*?)(?=\d{4}|$)", re.DOTALL)

# 找到所有标题行的位置
title_matches = list(title_pattern.finditer(content))
# 初始化存储结果的列表
stories_data = []

# 遍历 001 到 725，逐个定位标题行
for story_number in tqdm(range(1, number + 1), desc="处理故事", unit="个"):
    # 格式化编号为 3 位数（如 001, 002, ..., 725）
    story_number_str = f"{story_number:03d}"
    
    # 构造正则表达式，匹配当前编号的标题行
    current_title_pattern = re.compile(rf"{story_number_str}\s*[•.。](.*?)\n")
    title_match = current_title_pattern.search(content)
    
    if not title_match:
        # 如果没有找到当前编号的标题行，跳过
        continue

    # 提取标题
    story_title = title_match.group(1).strip()
    story_name = f"{story_number_str}{story_title}"

    # 提取地点信息
    location_match = location_pattern.search(content[title_match.end():])
    location = location_match.group(1).strip() if location_match else "NaN"

    # 提取故事内容
    if story_number < number:
        # 查找下一个标题行
        next_title_pattern = re.compile(rf"{story_number + 1:03d}\s*[•.。](.*?)\n")
        next_title_match = next_title_pattern.search(content[title_match.end():])
        if next_title_match:
            # 当前故事的内容是从当前标题开始到下一个标题之前
            story_content = content[title_match.start():title_match.end() + next_title_match.start()].strip()
        else:
            # 如果没有找到下一个标题行，则跳过
            continue
    else:
        # 最后一个故事的内容是从当前标题开始到文件末尾
        story_content = content[title_match.start():].strip()

    # 提取信息（优先级：讲述者 > 采录者 > 讲述并记录 > 翻译者）
    narrator_match = narrator_pattern.search(story_content)
    recorder_match = recorder_pattern.search(story_content)
    narrator_recorder_match = narrator_recorder_pattern.search(story_content)
    translator_match = translator_pattern.search(story_content)

    if narrator_match:
        info_text = narrator_match.group(1).strip()  # 提取“讲述者”之后的内容
        info_type = "讲述者"
        # 去除多余的标点符号和特殊字符，只保留汉字和数字
        info_text = re.sub(r"[^\u4e00-\u9fa5\d]", "", info_text)
        # 剔除讲述者信息
        story_content = story_content[:narrator_match.start()].strip()
    elif recorder_match:
        info_text = recorder_match.group(1).strip()  # 提取“采录者”之后的内容
        info_type = "采录者"
        # 进一步提取“采录者”所在行对应的内容
        recorder_line_match = re.search(r"采\s*录\s*者[^\w\s]*(.*)", info_text)
        if recorder_line_match:
            info_text = recorder_line_match.group(1).strip()
            # 去除多余的标点符号和特殊字符，只保留汉字和数字
            info_text = re.sub(r"[^\u4e00-\u9fa5\d]", "", info_text)
        else:
            info_text = "NaN"
        # 剔除采录者信息
        story_content = story_content[:recorder_match.start()].strip()
    elif narrator_recorder_match:
        info_text = narrator_recorder_match.group(1).strip()  # 提取“讲述并记录”之后的内容
        info_type = "讲述并记录"
        # 去除多余的标点符号和特殊字符，只保留汉字和数字
        info_text = re.sub(r"[^\u4e00-\u9fa5\d]", "", info_text)
        # 剔除讲述并记录信息
        story_content = story_content[:narrator_recorder_match.start()].strip()
    elif translator_match:
        info_text = translator_match.group(1).strip()  # 提取“翻译者”之后的内容
        info_type = "翻译者"
        # 去除多余的标点符号和特殊字符，只保留汉字和数字
        info_text = re.sub(r"[^\u4e00-\u9fa5\d]", "", info_text)
        # 剔除翻译者信息
        story_content = story_content[:translator_match.start()].strip()
    else:
        info_text = "NaN"
        info_type = "NaN"

    # 保存故事内容到单独的txt文件
    story_file_path = os.path.join(output_dir, f"{story_name}.txt")
    with open(story_file_path, "w", encoding="utf-8") as story_file:
        story_file.write(story_content)

    # 将故事信息添加到列表中
    stories_data.append({
        "故事编号": story_number_str,
        "故事标题": story_title,
        "地点信息": location,
        "采录者信息": info_text if info_type != "NaN" else "NaN"
    })

# 将故事信息保存到CSV文件
with open(csv_file_path, "w", encoding="utf-8", newline="") as csv_file:
    fieldnames = ["故事编号", "故事标题", "地点信息", "采录者信息"]
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(stories_data)

print(f"\n处理完成！结果已保存到 {output_dir} 目录中。")

处理故事: 100%|██████████| 641/641 [00:00<00:00, 1791.60个/s]


处理完成！结果已保存到 /Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250120OCR/内蒙古 目录中。


In [74]:
import os
import re
import csv

# 定义文件路径
input_file_path = "/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/data/中国民间故事集成 山东_青海_内蒙古_黑龙江/中国民间故事集成内蒙古卷.txt"
output_dir = "/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250120OCR/内蒙古"
csv_file_path = os.path.join(output_dir, "stories_info.csv")

number = 641
# 创建输出目录
os.makedirs(output_dir, exist_ok=True)

# 读取文件内容
with open(input_file_path, "r", encoding="utf-8") as file:
    content = file.read()

# 正则表达式匹配标题行（编号 + 可能存在的空格 + 分隔符 + 标题）
title_pattern = re.compile(r"(\d{3})\s*[•.。](.*?)\n")
# 正则表达式匹配采录者信息
recorder_pattern = re.compile(r"采\s*录\s*者[^\w\s]*(.*)")

# 找到所有标题行的位置
title_matches = list(title_pattern.finditer(content))

# 读取现有的 stories_info.csv 文件
with open(csv_file_path, "r", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    rows = list(reader)

# 遍历每个故事，重新清洗“采录者信息”列
for row in rows:
    story_number_str = row["故事编号"]
    
    # 构造正则表达式，匹配当前编号的标题行
    current_title_pattern = re.compile(rf"{story_number_str}\s*[•.。](.*?)\n")
    title_match = current_title_pattern.search(content)
    
    if not title_match:
        # 如果没有找到当前编号的标题行，跳过
        row["采录者信息"] = "NaN"
        continue

    # 提取故事内容
    if int(story_number_str) < number:
        # 查找下一个标题行
        next_title_pattern = re.compile(rf"{int(story_number_str) + 1:03d}\s*[•.。](.*?)\n")
        next_title_match = next_title_pattern.search(content[title_match.end():])
        if next_title_match:
            # 当前故事的内容是从当前标题开始到下一个标题之前
            story_content = content[title_match.start():title_match.end() + next_title_match.start()].strip()
        else:
            # 如果没有找到下一个标题行，则跳过
            row["采录者信息"] = "NaN"
            continue
    else:
        # 最后一个故事的内容是从当前标题开始到文件末尾
        story_content = content[title_match.start():].strip()

    # 查找“采录者”所在行
    recorder_match = recorder_pattern.search(story_content)
    if recorder_match:
        # 提取“采录者”所在行后面的内容
        recorder_info = recorder_match.group(1).strip()
        # 去除多余的标点符号和特殊字符，只保留汉字和数字
        recorder_info = re.sub(r"[^\u4e00-\u9fa5\d]", "", recorder_info)
        row["采录者信息"] = recorder_info
    else:
        row["采录者信息"] = "NaN"

# 将更新后的数据写回 stories_info.csv 文件
with open(csv_file_path, "w", encoding="utf-8", newline="") as csvfile:
    fieldnames = ["故事编号", "故事标题", "地点信息", "采录者信息"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(rows)

print(f"\n“采录者信息”列清洗完成！结果已保存到 {csv_file_path} 文件中。")


“采录者信息”列清洗完成！结果已保存到 /Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250120OCR/内蒙古/stories_info.csv 文件中。


正常版的清洗采录者信息

In [67]:
# 读取 stories_info.csv 文件
with open(csv_file_path, "r", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    rows = list(reader)

# 处理每一行的采录者信息
for row in rows:
    recorder_info = row["采录者信息"]
    if recorder_info != "NaN":
        # 如果采录者信息不为空，进一步提取“采录者”或“讲述并记录”之后的内容
        recorder_match = re.search(r"(?:采\s*录\s*者|讲\s*述\s*并\s*记\s*录)[^\w\s]*(.*)", recorder_info)
        if recorder_match:
            # 提取“采录者”或“讲述并记录”之后的内容
            recorder_info = recorder_match.group(1).strip()
            # 去除多余的标点符号和特殊字符，只保留汉字和数字
            recorder_info = re.sub(r"[^\u4e00-\u9fa5\d]", "", recorder_info)
            row["采录者信息"] = recorder_info

# 将处理后的数据写回 stories_info.csv 文件
with open(csv_file_path, "w", encoding="utf-8", newline="") as csvfile:
    fieldnames = ["故事编号", "故事标题", "地点信息", "采录者信息"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(rows)

print(f"\n采录者信息处理完成！结果已保存到 {csv_file_path} 文件中。")


采录者信息处理完成！结果已保存到 /Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250120OCR/内蒙古/stories_info.csv 文件中。


清洗故事文本内容（去除脚注、段内换行等），这个代码要运行两次

In [71]:
import os
import re

# 定义输出目录
output_dir = "/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250120OCR/内蒙古"

# 正则表达式匹配带圆圈编号的行（包括中英文圆圈编号）
circle_pattern = re.compile(r"^\s*[①-⑳ⓐ-ⓩa-z]\.?\s*.*")

# 遍历所有故事的txt文件
for file_name in os.listdir(output_dir):
    if file_name.endswith(".txt"):
        file_path = os.path.join(output_dir, file_name)
        
        # 读取文件内容
        with open(file_path, "r", encoding="utf-8") as file:
            lines = file.readlines()

        # 清洗数据
        cleaned_lines = []

        for line in lines:
            # 1. 删除带圆圈编号的行（包括中英文圆圈编号）
            if circle_pattern.match(line):
                continue  # 跳过带圆圈编号的行

            # 2. 删除单独成一行的数字（OCR 页码）及其前后的空行
            if re.match(r"^\s*\d+\s*$", line):
                continue  # 跳过单独成一行的数字
            if len(cleaned_lines) > 0 and re.match(r"^\s*\d+\s*$", cleaned_lines[-1]) and line.strip() == "":
                continue  # 跳过单独成一行数字后的空行
            if len(cleaned_lines) > 0 and re.match(r"^\s*\d+\s*$", line) and cleaned_lines[-1].strip() == "":
                cleaned_lines.pop()  # 删除单独成一行数字前的空行

            # 将当前行加入清洗后的列表
            cleaned_lines.append(line)

        # 进一步处理段落内的换行符
        final_lines = []
        i = 0
        while i < len(cleaned_lines):
            current_line = cleaned_lines[i]
            if i < len(cleaned_lines) - 1:
                next_line = cleaned_lines[i + 1]
                # 如果前一行以空格开头，后一行以文字开头，说明属于同一段落
                if current_line.startswith(" ") and not next_line.startswith(" "):
                    final_lines.append(current_line.rstrip() + next_line)
                    i += 2  # 跳过下一行，因为已经合并
                    continue
                # 如果两行都以文字开头，说明属于同一段落
                elif not current_line.startswith(" ") and not next_line.startswith(" "):
                    final_lines.append(current_line.rstrip() + next_line)
                    i += 2  # 跳过下一行，因为已经合并
                    continue
                # 如果前一行以文字开头，后一行以空格开头，则不做操作
                else:
                    final_lines.append(current_line)
                    i += 1
            else:
                final_lines.append(current_line)
                i += 1

        # 保存清洗后的内容
        with open(file_path, "w", encoding="utf-8") as file:
            file.writelines(final_lines)

print("数据清洗完成！")

数据清洗完成！


按照全部总体的故事数量重新编号，并清洗stories_info.csv这个表（去除特殊字符）

In [72]:
import os
import re
import csv

# 定义目录和文件路径
output_dir = "/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250120OCR/内蒙古"
csv_path = os.path.join(output_dir, "stories_info.csv")

# 初始化编号
start_number = 19172

# 1. 按顺序遍历txt文件，重命名文件
txt_files = [f for f in os.listdir(output_dir) if f.endswith(".txt")]
txt_files.sort()  # 按文件名排序

for idx, file_name in enumerate(txt_files):
    # 构造新文件名
    new_file_name = f"{start_number + idx}__{file_name}"
    old_file_path = os.path.join(output_dir, file_name)
    new_file_path = os.path.join(output_dir, new_file_name)
    
    # 重命名文件
    os.rename(old_file_path, new_file_path)

print("文件重命名完成！")

# 2. 读取stories_info.csv，修改故事标题列
def clean_text(text):
    """删除标点符号和特殊字符，只保留汉字和数字"""
    return re.sub(r"[^\u4e00-\u9fa5\d]", "", text)

# 读取CSV文件
rows = []
with open(csv_path, "r", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    fieldnames = reader.fieldnames  # 获取表头
    for row in reader:
        rows.append(row)

# 修改故事标题列
for idx, row in enumerate(rows):
    # 构造新的故事标题
    story_number = row["故事编号"].zfill(3)  # 补全为3位数字
    new_title = f"{start_number + idx}__{story_number}{row['故事标题']}"
    row["故事标题"] = new_title

    # 清洗地点信息和采录者信息列
    row["地点信息"] = clean_text(row["地点信息"])
    row["采录者信息"] = clean_text(row["采录者信息"])

# 写回CSV文件
with open(csv_path, "w", encoding="utf-8", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(rows)

print("CSV文件修改完成！")

文件重命名完成！
CSV文件修改完成！


修改info表的内容，使得和之前“地图匹配”表一致

In [81]:
import pandas as pd

# 定义地区变量
region = "黑龙江"

# 读取CSV文件
file_path = f'/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250120OCR/{region}/stories_info.csv'
df = pd.read_csv(file_path)

# 在地点信息一列每个单元格取值前加入地区变量
df['地点信息'] = region + df['地点信息'].astype(str)

# 新生成一列“new_name”取值为“故事标题”里的中文
df['new_name'] = df['故事标题'].str.extract('([\u4e00-\u9fff]+)')  # 提取中文

# 新生成一列area_group为地区变量
df['area_group'] = region

# 保存修改后的数据到新的CSV文件（可选）
output_file_path = f'/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250120OCR/{region}/stories_info_modified.csv'
df.to_csv(output_file_path, index=False, encoding='utf-8-sig')


下面就开始合并数据到地图匹配总表中

In [87]:
import pandas as pd

# 定义文件路径
csv_file_path = '/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250120OCR/内蒙古641/250120stories_info_neimenggu.csv'
excel_file_path = '/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250120地图匹配_v6.xlsx'
output_excel_path = '/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250120地图匹配_v6.xlsx'

# 读取CSV文件
df_csv = pd.read_csv(csv_file_path)

# 读取Excel文件
df_excel = pd.read_excel(excel_file_path)

# 找到两个表中相同的列名
common_columns = set(df_csv.columns) & set(df_excel.columns)

# 如果存在相同列名
if common_columns:
    # 提取CSV表中与Excel表相同列名的数据
    df_csv_common = df_csv[list(common_columns)]

    # 在CSV表的数据中添加Excel表中独有的列，并填充空值
    for column in df_excel.columns:
        if column not in common_columns:
            df_csv_common[column] = None  # 添加Excel独有的列，并填充空值

    # 将CSV表中的数据追加到Excel表的下方
    merged_df = pd.concat([df_excel, df_csv_common], ignore_index=True)

    # 保存合并后的数据到新的Excel文件
    merged_df.to_excel(output_excel_path, index=False)
    print(f"合并后的数据已保存到: {output_excel_path}")
else:
    print("两个表没有相同的列名，无法合并。")

/var/folders/w5/_ylk19595zl7_649y60st5680000gn/T/ipykernel_50799/2683165112.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_df = pd.concat([df_excel, df_csv_common], ignore_index=True)


合并后的数据已保存到: /Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/250120地图匹配_v6.xlsx
